In [1]:
import importlib
import pandas as pd
import torch
import numpy as np
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from datasets import Dataset

from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from Project.utils.storage import truth_db as t_db
from Project.utils.storage import bluesky_db as b_db

importlib.reload(t_db)
importlib.reload(b_db)

truth_db = t_db.SQLiteTruthSaver(db_name='../db/truthsocial.db')
bluesky_db = b_db.SQLiteBlueSkySaver(db_name='../db/bluesky.db')

C:\Users\Filippo Corti\miniconda3\envs\SocialMediaMining\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# THE PLAN:

# 1. Label some comments from both socials, with Gemini

# 2. Fine tune a HuggingFace model

# 3. Run the model, don't care about performance honestly


In [2]:
truth_db.cursor.execute("""
SELECT content, gemini_label
FROM Posts NATURAL JOIN CommentAnalysis
""")
truth_data = truth_db.cursor.fetchall()

bluesky_db.cursor.execute("""
SELECT content, gemini_label
FROM Posts NATURAL JOIN CommentAnalysis
""")
bluesky_data = bluesky_db.cursor.fetchall()

data = truth_data + bluesky_data

df = pd.DataFrame(data, columns=['content', 'label'])
df['label'] = [{'Neutral': 0, 'Republican': 1, 'Democratic': 2}.get(l) for l in df['label']]

In [3]:
df

,content,label
0,<p>Score!</p>,0
1,<p>The 170 can make maple syrup!</p>,0
2,<p>Exactly and they’re not just in one party !...,0
3,<p>why dont you go away</p>,0
4,<p>Democrats are destroying America. Why? Is i...,1
...,...,...
4395,California led the way on this one. Don’t forg...,0
4396,📌I see Gavin Newsom saying that a man who is r...,2
4397,That's the point. So there's nothing left but ...,2
4398,A stroller does not have to cost a thousand bu...,0


In [4]:
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['label']  # Ensures balanced splits
)

print(f"Train size: {len(train_df)}, Test size: {len(test_df)}")

Train size: 3520, Test size: 880


In [3]:
# Model

model_name = "matous-volf/political-leaning-politics"
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3  # 0, 1, 2
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "launch/POLITICS",
    use_fast=False
)


def tokenize_function(data):
    return tokenizer(
        data['content'],
        truncation=True,
        padding=True,
        max_length=512
    )

In [6]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


Map: 100%|██████████| 880/880 [00:00<00:00, 1227.19 examples/s]


In [7]:
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_df['label']),
    y=train_df['label']
)
class_weights = torch.tensor(class_weights, dtype=torch.float)

print(f"Class weights: {class_weights}")

Class weights: tensor([0.5090, 2.1295, 1.7671])


In [8]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_score(labels, predictions)
    report = classification_report(labels, predictions, output_dict=True)

    return {
        'accuracy': accuracy,
        'f1_macro': report['macro avg']['f1-score'],
        'f1_weighted': report['weighted avg']['f1-score']
    }


training_args = TrainingArguments(
    output_dir='./results',
    logging_dir='./logs',
    num_train_epochs=3,
    warmup_steps=100,  # Start with a smaller LR
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,  # Regularization
    eval_steps=200,  # How often it does evaluation
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,  # Needed if metric is f1 instead of loss
    report_to=None,
)

model.gradient_checkpointing_enable()

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


C:\Users\Filippo Corti\AppData\Local\Temp\ipykernel_31596\152238528.py:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [9]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Weighted
200,No log,0.673560,0.718182,0.555087,0.678162
400,No log,0.860501,0.661364,0.617458,0.678215
600,0.738700,0.834527,0.745455,0.687885,0.751763
800,0.738700,0.772857,0.726136,0.682134,0.737337
1000,0.501100,0.971516,0.709091,0.676428,0.721736
1200,0.501100,1.080729,0.692045,0.651945,0.705056


TrainOutput(global_step=1320, training_loss=0.5616306940714518, metrics={'train_runtime': 1225.9534, 'train_samples_per_second': 8.614, 'train_steps_per_second': 1.077, 'total_flos': 2778477691207680.0, 'train_loss': 0.5616306940714518, 'epoch': 3.0})

In [10]:
# Evaluate the model
results = trainer.evaluate()
print("Evaluation Results:")
for key, value in results.items():
    print(f"{key}: {value}")

Evaluation Results:
eval_loss: 0.8345268964767456
eval_accuracy: 0.7454545454545455
eval_f1_macro: 0.6878851613264981
eval_f1_weighted: 0.7517625157383435
eval_runtime: 18.5469
eval_samples_per_second: 47.447
eval_steps_per_second: 5.931
epoch: 3.0


In [13]:
# Save model
model.save_pretrained('../data/bluesky_truth_model')
tokenizer.save_pretrained('../data/bluesky_truth_model')

('../data/bluesky_truth_model\\tokenizer_config.json',
 '../data/bluesky_truth_model\\special_tokens_map.json',
 '../data/bluesky_truth_model\\vocab.json',
 '../data/bluesky_truth_model\\merges.txt',
 '../data/bluesky_truth_model\\added_tokens.json')

In [5]:
# Load model
model_path = '../data/bluesky_truth_model'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2212, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
      

In [8]:
# Load full datasets

truth_db.cursor.execute("""
SELECT id, content
FROM Posts
""")
truth_data = truth_db.cursor.fetchall()
truth_df = pd.DataFrame(truth_data, columns=['id', 'content'])

bluesky_db.cursor.execute("""
SELECT id, content
FROM Posts
""")
bluesky_data = bluesky_db.cursor.fetchall()
bluesky_df = pd.DataFrame(bluesky_data, columns=['id', 'content'])


In [9]:
truth_df

,id,content
0,114197083930867087,"<p>Canadian Furniture Maker Closes B.C. Plant,..."
1,114197087215443716,<p>Smart move!!!</p>
2,114197089158254652,<p>Score!</p>
3,114197090765627568,<p>Need to make it law that states who receive...
4,114197093134399024,<p>Welcome future hard working American🤗</p>
...,...,...
73870,114387866091101141,<p>Tariff away sir! I want to see the IRS abol...
73871,114387907123632449,"<p>The tariffs were NOT reciprocal, and Trump,..."
73872,114387964495138238,<p>&quot;Protective Agricultural Standards&quo...
73873,114387966057950562,<p>How?</p>


In [11]:
bluesky_df

,id,content
0,bafyreid6xubyttiiz5waygebjoeph7252n3ilrh4b342o...,China’s reaction to the US Tariffs
1,bafyreihxt34udnghza4ncdfpod7uhiprf6fnxi7zpd6uj...,JUST IN: Karoline Leavitt now says there will ...
2,bafyreicqxfk3sbk6hr5nigt3hsdr3a7q2nt2a5puxdlh4...,MOTHERFUCKER NO NOT MORE TARIFFS
3,bafyreidbjqqkxs56hccgwzgb3hk3mrhkijpj4wtg4jxwv...,A dozen states sued President Trump over his t...
4,bafyreicqnjgh4pr3p5pxdbmvsqss2gsiaqav4mlim4es4...,"I'm raising tariffs, I'm cutting tariffs. I'm ..."
...,...,...
20105,bafyreifafxhr7iwiyj7tu3pspygwqjlwudhpmsi2dq2j5...,JUST IN: A federal appeals court has paused th...
20106,bafyreieo2pueka5vop65putamluewsfvovhat6hzhdxme...,A US court has blocked President Donald Trump'...
20107,bafyreiegfyxm46iiuuu5wvinrqzvnk6zylzcoyd53y536...,What? Did they think we would get tired alread...
20108,bafyreichf7lbc5se3abia6o3emjoij6gvzodwxat3j7fh...,Let's be honest. When Biden started banging on...


In [17]:
# Predict Bluesky data

all_predictions = list()

# Process in batches of size 100
batch_size = 100
for i in range(0, len(bluesky_df), batch_size):
    texts = bluesky_df['content'][i:i + batch_size].tolist()
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt").to('cuda')

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits.argmax(dim=-1).cpu().numpy()

    all_predictions.extend(predictions)
    print(f"{i}/{len(bluesky_df)}")

bluesky_df['predicted_label'] = all_predictions

0/20110
100/20110
200/20110
300/20110
400/20110
500/20110
600/20110
700/20110
800/20110
900/20110
1000/20110
1100/20110
1200/20110
1300/20110
1400/20110
1500/20110
1600/20110
1700/20110
1800/20110
1900/20110
2000/20110
2100/20110
2200/20110
2300/20110
2400/20110
2500/20110
2600/20110
2700/20110
2800/20110
2900/20110
3000/20110
3100/20110
3200/20110
3300/20110
3400/20110
3500/20110
3600/20110
3700/20110
3800/20110
3900/20110
4000/20110
4100/20110
4200/20110
4300/20110
4400/20110
4500/20110
4600/20110
4700/20110
4800/20110
4900/20110
5000/20110
5100/20110
5200/20110
5300/20110
5400/20110
5500/20110
5600/20110
5700/20110
5800/20110
5900/20110
6000/20110
6100/20110
6200/20110
6300/20110
6400/20110
6500/20110
6600/20110
6700/20110
6800/20110
6900/20110
7000/20110
7100/20110
7200/20110
7300/20110
7400/20110
7500/20110
7600/20110
7700/20110
7800/20110
7900/20110
8000/20110
8100/20110
8200/20110
8300/20110
8400/20110
8500/20110
8600/20110
8700/20110
8800/20110
8900/20110
9000/20110
9100/20110


In [19]:
bluesky_df

,id,content,predicted_label
0,bafyreid6xubyttiiz5waygebjoeph7252n3ilrh4b342o...,China’s reaction to the US Tariffs,0
1,bafyreihxt34udnghza4ncdfpod7uhiprf6fnxi7zpd6uj...,JUST IN: Karoline Leavitt now says there will ...,0
2,bafyreicqxfk3sbk6hr5nigt3hsdr3a7q2nt2a5puxdlh4...,MOTHERFUCKER NO NOT MORE TARIFFS,0
3,bafyreidbjqqkxs56hccgwzgb3hk3mrhkijpj4wtg4jxwv...,A dozen states sued President Trump over his t...,2
4,bafyreicqnjgh4pr3p5pxdbmvsqss2gsiaqav4mlim4es4...,"I'm raising tariffs, I'm cutting tariffs. I'm ...",2
...,...,...,...
20105,bafyreifafxhr7iwiyj7tu3pspygwqjlwudhpmsi2dq2j5...,JUST IN: A federal appeals court has paused th...,2
20106,bafyreieo2pueka5vop65putamluewsfvovhat6hzhdxme...,A US court has blocked President Donald Trump'...,2
20107,bafyreiegfyxm46iiuuu5wvinrqzvnk6zylzcoyd53y536...,What? Did they think we would get tired alread...,2
20108,bafyreichf7lbc5se3abia6o3emjoij6gvzodwxat3j7fh...,Let's be honest. When Biden started banging on...,0


In [22]:
# Predict Truthsocial data

all_predictions = list()

# Process in batches of size 50
batch_size = 50
for i in range(0, len(truth_df), batch_size):
    texts = truth_df['content'][i:i + batch_size].tolist()
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt").to('cuda')

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits.argmax(dim=-1).cpu().numpy()

    all_predictions.extend(predictions)
    print(f"{i}/{len(truth_df)}")

truth_df['predicted_label'] = all_predictions

0/73875
50/73875
100/73875
150/73875
200/73875
250/73875
300/73875
350/73875
400/73875
450/73875
500/73875
550/73875
600/73875
650/73875
700/73875
750/73875
800/73875
850/73875
900/73875
950/73875
1000/73875
1050/73875
1100/73875
1150/73875
1200/73875
1250/73875
1300/73875
1350/73875
1400/73875
1450/73875
1500/73875
1550/73875
1600/73875
1650/73875
1700/73875
1750/73875
1800/73875
1850/73875
1900/73875
1950/73875
2000/73875
2050/73875
2100/73875
2150/73875
2200/73875
2250/73875
2300/73875
2350/73875
2400/73875
2450/73875
2500/73875
2550/73875
2600/73875
2650/73875
2700/73875
2750/73875
2800/73875
2850/73875
2900/73875
2950/73875
3000/73875
3050/73875
3100/73875
3150/73875
3200/73875
3250/73875
3300/73875
3350/73875
3400/73875
3450/73875
3500/73875
3550/73875
3600/73875
3650/73875
3700/73875
3750/73875
3800/73875
3850/73875
3900/73875
3950/73875
4000/73875
4050/73875
4100/73875
4150/73875
4200/73875
4250/73875
4300/73875
4350/73875
4400/73875
4450/73875
4500/73875
4550/73875
4600/73875


In [23]:
truth_df

,id,content,predicted_label
0,114197083930867087,"<p>Canadian Furniture Maker Closes B.C. Plant,...",0
1,114197087215443716,<p>Smart move!!!</p>,0
2,114197089158254652,<p>Score!</p>,0
3,114197090765627568,<p>Need to make it law that states who receive...,0
4,114197093134399024,<p>Welcome future hard working American🤗</p>,1
...,...,...,...
73870,114387866091101141,<p>Tariff away sir! I want to see the IRS abol...,2
73871,114387907123632449,"<p>The tariffs were NOT reciprocal, and Trump,...",2
73872,114387964495138238,<p>&quot;Protective Agricultural Standards&quo...,1
73873,114387966057950562,<p>How?</p>,0


In [25]:
# Next:
# Save predictions in DBs
# Check if we can plot anything about the trainer's performance

update_data = [(label, id) for id, label in zip(truth_df['id'], truth_df['predicted_label'])]

# Execute all at once
cursor = truth_db.cursor
cursor.executemany(
    "UPDATE CommentAnalysis SET label = ? WHERE id = ?",
    update_data
)
truth_db.conn.commit()


In [26]:
update_data = [(label, id) for id, label in zip(bluesky_df['id'], bluesky_df['predicted_label'])]

# Execute all at once
cursor = bluesky_db.cursor
cursor.executemany(
    "UPDATE CommentAnalysis SET label = ? WHERE id = ?",
    update_data
)
bluesky_db.conn.commit()